# Trabajo Practico 1

Ejercios a entregar: 21 7 20 23 30 57 60

##Imports y datasets

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
import collections
from datetime import datetime
from string import punctuation
from collections import Counter
from scipy.special import rel_entr
from scipy.special import entr
from nltk import word_tokenize
from nltk import trigrams
from nltk.util import ngrams
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from sklearn.feature_extraction.text import CountVectorizer


In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
id='12KNPiAF8AsGTLGRSpMUw2XdwYXwSosr3'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('geo_tags.csv')
geo_tags = pd.read_csv('geo_tags.csv')

In [ ]:
id='1Nxk1FhbEpwAqM9070I-0ESVa4wKEVMaB'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('contents.csv')
contents = pd.read_csv('contents.csv')

In [ ]:
id= '1RILGHMiAuB3n3Nc1n1cHshOR4QkQLDmb'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('languages.csv')
languages = pd.read_csv('languages.csv')

In [ ]:
id='1E1HtTbXVdwJJW7Z7XeyMf9H-GQVgD0OK'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('logs.csv')
logs = pd.read_csv('logs.csv',dtype={'comment':"string"})

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
id='14B07LGSE60-P423xMrTVQAzGvLXsqm_2'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('contents_sample.csv')
contents_sample = pd.read_csv('contents_sample.csv')

## 21) ¿Dónde está la referencia geográfica más repetida en la tierra de toda la Wikipedia
Español?

Asumo que la referencia geografica es una ubicacion con una respectiva latitud y longitud y busco la latitud y longitud mas repetida

In [ ]:
geo_tags_earth = geo_tags[geo_tags['gt_globe']=='earth']
ordenado_por_referencias = pd.DataFrame(geo_tags_earth.value_counts(['gt_lat','gt_lon']))
ordenado_por_referencias.reset_index(inplace = True)
mas_repetida = ordenado_por_referencias.iloc[0]
print(f"La referencia geografica mas repetida esta en latitud: {mas_repetida['gt_lat']}, longitud: {mas_repetida['gt_lon']} y se repite un total de {mas_repetida[0]} veces")

La referencia geografica mas repetida esta en latitud: -30.5, longitud: -64.5 y se repite un total de 255.0 veces


## 7) La antigüedad promedio de la última edición de los artículos cuyo título contenga tu apellido (si no hay, tu nombre y si tampoco hay usa Cafferata)

Veo cuantos titulos son nulos y los saco del dataframe

In [ ]:
dfNulos = pd.DataFrame({'Cantidad de nulos': contents.isnull().sum(), 'Porcentaje de nulos': (contents.isnull().mean() * 100)})
dfNulos

,Cantidad de nulos,Porcentaje de nulos
title,4,0.000097
id,0,0.000000
namespace,0,0.000000
revision_id,0,0.000000
parent_revision,1483218,35.894461
revision_timestamp,0,0.000000
revisor_username,193287,4.677622
revisor_id,193287,4.677622
revisor_ip,3938878,95.322402
revisor_comment,469224,11.355406


In [ ]:
contents.dropna(subset=['title'],inplace=True)

Me quedo con los elementos del dataframe que contienen el apellido y con las columnas que voy a utilizar, luego para hallar la antiguedad resto la fecha actual con la fecha que fue publicado el contenido y tomo el promedio

In [ ]:
contents_reducido = contents.loc[contents['title'].str.contains("Cafferata"), ['title','revision_timestamp']].reset_index(drop=True) 
fecha_actual = pd.to_datetime("now",utc=True)
contents_reducido['antiguedad'] = fecha_actual - pd.to_datetime(contents_reducido.revision_timestamp)
promedio_antiguedad = contents_reducido.antiguedad.mean()
print("The average is", promedio_antiguedad)

The average is 980 days 04:26:12.552664672


## 23)¿Qué porcentaje de los contenidos contienen a su mismo título en el texto?

In [ ]:
contents_sample.dropna(subset=['text'],inplace=True)

Paso todo a minusculas y armo una serie booleana con los que tienen al titulo en su texto, despues tomo porcentaje

In [ ]:
titulo_en_texto = contents_sample.apply(lambda row: row['title'].lower() in row['text'].lower(), axis=1)
porcentaje = titulo_en_texto.mean()*100
print("El porcentaje es ", porcentaje)

El porcentaje es  38.3809293320426


## 30)  La 3-upla de palabras más común en los comentarios de los logs

Saque los comentarios nulos ya que logicamente no servian, luego tome una sample de comentarios de 500000 por falta de RAM. Hice mi propia lista para poder tokenizar los textos, sacando algunos simbolos y despues aplique word tokenize para tener las palabras separadas

In [ ]:
comments = pd.DataFrame(logs.comment)
comments.dropna(subset=['comment'],inplace=True)

In [ ]:
serie_sample = comments['comment'].sample(500000)

In [ ]:
lista = ["[", "]", "'", "{", "}", "#", "|", ":", ";", "&", "*", ".", "(", ")", "=", "%", "$", "/", "«", "»", ",", "``", "--", "-", ">", "<", "?","``"]
for x in lista:
  serie_sample = serie_sample.str.replace(x, " ")


Tokenizo la serie para tener todos los comentarios separados por palabras

In [ ]:
#nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
serie_tokenizada = serie_sample.apply(word_tokenize)
dataframe_tokenizado = pd.DataFrame(serie_tokenizada)

Busco los ngramas y los coloco en un Counter, y de ahi busco el mas repetido

In [ ]:
contador = Counter()
for x in dataframe_tokenizado['comment']:
  if len(x)>1:
    contador.update(list(ngrams(x,3)))
print("La triupla mas comun es", contador.most_common(1))

La triupla mas comun es [(('el', 'contenido', 'era'), 106512)]


## 20)¿Cuál es el segundo contenido con más referencias geográficas asignadas?

Me quedo unicamente con los ids de geo tags que estan en contents, luego veo los ids de referencias geograficas en contenidos que mas se repitan y devuelvo el titulo del mas repetido

In [ ]:
geo_tags_en_contents = geo_tags['gt_page_id'].isin(contents['id'])
geo_tags_filtrado = geo_tags[geo_tags_en_contents]

In [ ]:
id_contenidos = geo_tags_filtrado.value_counts(['gt_page_id'])
id_mas_contenidos = id_contenidos.index[1][0]
mas_repetido = contents.loc[contents['id']==id_mas_contenidos,'title'].reset_index(drop=True)
print("El segundo contenido con mas referencias geograficas es",mas_repetido[0])

El segundo contenido con mas referencias geograficas es Wikiproyecto:Madrid/Artículos/Wikidata


## 57). Calcule la probabilidad de que una acción en general se realice según día de la semana. Calcule también para los días de la semana la probabilidad de que la última edición de un contenido sea realizada ese día. Calcule la entropía de ambas y la divergencia de Kullback Leibler entre ellas.

Saco las acciones nulas, y creo una nueva columna que contenga en que dia de la semana se realizo cada accion

In [ ]:
logs.dropna(subset=['action'],inplace=True)

In [ ]:
logs['timestamp'] = pd.to_datetime(logs['timestamp']) 
logs['weekday'] = logs.agg({'timestamp':datetime.weekday})  


Cuento cuantas acciones se realizaron por dia de semana

Creo una nueva columna que contenga la probabilidad de que cada accion se realice en cada dia de la semana, divido las acciones de cada dia por el total de acciones

In [ ]:
ediciones_por_dia = logs['weekday'].value_counts()
probabilidad_acciones = (ediciones_por_dia)/np.asarray(ediciones_por_dia.sum())
probabilidad_acciones

6    0.159515
1    0.148865
2    0.148164
0    0.145012
3    0.144631
4    0.132325
5    0.121488
Name: weekday, dtype: float64

Ahora calculo la otra probabilidad que piden de manera analoga

In [ ]:
contents['revision_timestamp'] = pd.to_datetime(contents['revision_timestamp'])  
contents['weekday'] = contents.agg({'revision_timestamp':datetime.weekday})     

Veo cuantas ediciones se hicieron por dia y saco la probabilidad

In [ ]:
ediciones_por_dia = contents['weekday'].value_counts()
probabilidad_ediciones = (ediciones_por_dia)/np.asarray(ediciones_por_dia.sum())
probabilidad_ediciones

1    0.154101
0    0.151421
3    0.149396
2    0.149237
6    0.132654
5    0.132125
4    0.131066
Name: weekday, dtype: float64

In [ ]:
divergencia = rel_entr(probabilidad_ediciones, probabilidad_acciones)
print ("La divergencia es",divergencia.sum())

La divergencia es 0.0031711468019571983


In [ ]:
divergencia = rel_entr(probabilidad_acciones, probabilidad_ediciones)
print ("La divergencia es",divergencia.sum())

La divergencia es 0.0033069182785994966


In [ ]:
entropia_acciones = entr(probabilidad_acciones)
print ("La entropia de las acciones es",entropia_acciones.sum())

La entropia de las acciones es 1.9426429437598134


In [ ]:
entropia_ediciones = entr(probabilidad_ediciones)
print ("La entropia de las ediciones es",entropia_ediciones.sum())

La entropia de las ediciones es 1.943651772615871


## 60) . Si decimos que un usuario sabe un idioma cuando tiene un nivel de babel mayor o igual a 1, para aquellos que editaron una de las versiones actuales del contenido, ¿Cuál es la cantidad de agradecimientos promedio que reciben en función de los idiomas que saben?

Como no especifica que pasa para los usuarios que tienen N(nativo) en el nivel de babel, lo tomo como que tambien saben ese idioma, asique los agregue, luego filtro los que tienen nivel en cero y me quedo solamente que esten en contents

In [ ]:
languages.loc[languages['babel_level'] == 'N', 'babel_level'] = 1
usuarios_con_idioma = languages.loc[(languages[('babel_level')]).astype('int')>0,:]
esta_en_contents = usuarios_con_idioma['babel_user'].isin(contents['revisor_id'])
usuarios_en_contents = usuarios_con_idioma[esta_en_contents]                

Agrupo estos usuarios y cuento cuantos idiomas sabe cada uno

In [ ]:
usuarios_agrupados_por_lenguaje = usuarios_en_contents.groupby('babel_user').agg({'babel_lang':'count'})  
usuarios_agrupados_por_lenguaje.rename(columns={'babel_lang': 'cantidad_de_idiomas'},inplace=True) 
usuarios_agrupados_por_lenguaje

,cantidad_de_idiomas
babel_user,
30,9
38,2
311,6
322,3
373,7
...,...
6306637,1
6315666,1
6318249,3


Ahora me quedo con todos los usuarios que hayan hecho al menos una revision en contents

In [ ]:
contents_reducido = contents[['revisor_id','revisor_username']].copy()
contents_reducido.dropna(subset=['revisor_id'],inplace=True)
contents_reducido.drop_duplicates(inplace=True)
contents_reducido       

,revisor_id,revisor_username
0,1708233.0,Metrónomo
2,4642283.0,Traitrich
3,5338508.0,Lojwe
4,3615366.0,Jialxv
5,689866.0,PatriHorrillo
...,...,...
4131723,6332400.0,Templajaen
4131771,6291332.0,Islasazules
4131957,6331508.0,BlockchainWolf
4132053,6252228.0,Pommelito crazy


Junto todos los que hayan hecho una revision con los que tienen nivel de babel mayor a 1 o nativos

In [ ]:
contents_con_idiomas = pd.merge(left=contents_reducido, right=usuarios_agrupados_por_lenguaje, left_on='revisor_id', right_on='babel_user')
contents_con_idiomas

,revisor_id,revisor_username,cantidad_de_idiomas
0,4642283.0,Traitrich,5
1,3615366.0,Jialxv,4
2,689866.0,PatriHorrillo,3
3,6048511.0,LOLAOMI,1
4,936194.0,Jkbw,5
...,...,...,...
4626,1495557.0,Rafamol,6
4627,6116537.0,Farmengol,2
4628,5130327.0,Dhneil,7
4629,5913066.0,Dr Salvus,3


Filtro todos los agradecimientos recibidos y me fijo que todos se identifiquen por nombre de usuario y no por IP

In [ ]:
filtrados_por_agradecimiento = logs.loc[logs[('action')]=="thank",:]
usuarios_agradecidos = filtrados_por_agradecimiento['title']
contienen_usuario = usuarios_agradecidos.str.contains("Usuario")
contienen_usuario.value_counts 


<bound method IndexOpsMixin.value_counts of 5089429     True
5089430     True
5089443     True
5089554     True
5089560     True
            ... 
12188876    True
12188903    True
12188905    True
12188919    True
12188922    True
Name: title, Length: 241948, dtype: object>

In [ ]:
usuarios_agradecidos = usuarios_agradecidos.str.replace("Usuario:", "")

Armo un dataframe con todos los agradecimientos que recibieron los usuarios

In [ ]:
df_usuarios_agradecidos = usuarios_agradecidos.to_frame(name = "recibio_agradecimiento").reset_index(drop=True) 
df_usuarios_agradecidos

,recibio_agradecimiento
0,Isha
1,Osado
2,Fabrice Florin
3,Sala Zero
4,StockZXD
...,...
241943,Tector
241944,Latothy99
241945,JulioW4rrior00
241946,Alpinu


In [ ]:
prueba = df_usuarios_agradecidos.value_counts()
usuarios_agradecidos_con_cantidad = pd.DataFrame(prueba, columns=['cantidad_de_agradecimientos'])
usuarios_agradecidos_con_cantidad.reset_index(inplace = True)
usuarios_agradecidos_con_cantidad

,recibio_agradecimiento,cantidad_de_agradecimientos
0,Ontzak,4975
1,Marcelo,4372
2,Geom,2793
3,Fixertool,2778
4,Taichi,2747
...,...,...
22377,Luis Eugenio Panza,1
22378,Luis Crespí de Valldaura,1
22379,Luis Cheetos,1
22380,Luis Cedeño Rodríguez,1


In [ ]:
contents_con_idiomas_y_agradecimientos = pd.merge(left=contents_con_idiomas, right=usuarios_agradecidos_con_cantidad, left_on='revisor_username', right_on='recibio_agradecimiento', how = 'left')
contents_con_idiomas_y_agradecimientos.fillna(0, inplace = True)
contents_con_idiomas_y_agradecimientos 

,revisor_id,revisor_username,cantidad_de_idiomas,recibio_agradecimiento,cantidad_de_agradecimientos
0,4642283.0,Traitrich,5,Traitrich,30.0
1,3615366.0,Jialxv,4,Jialxv,100.0
2,689866.0,PatriHorrillo,3,PatriHorrillo,198.0
3,6048511.0,LOLAOMI,1,0,0.0
4,936194.0,Jkbw,5,Jkbw,1247.0
...,...,...,...,...,...
4626,1495557.0,Rafamol,6,0,0.0
4627,6116537.0,Farmengol,2,0,0.0
4628,5130327.0,Dhneil,7,Dhneil,1.0
4629,5913066.0,Dr Salvus,3,0,0.0


Una vez que tengo los agradecimientos por usuario y la cantidad de idiomas que hablo saco el promedio de agradecimientos que reciben por cantidad de idiomas que saben

In [ ]:
promedio = contents_con_idiomas_y_agradecimientos.groupby('cantidad_de_idiomas').agg({'cantidad_de_agradecimientos':'mean'})
promedio.rename(columns={'cantidad_de_agradecimientos': 'cantidad_de_agradecimientos_promedio'},inplace=True)
promedio

,cantidad_de_agradecimientos_promedio
cantidad_de_idiomas,
1,10.028986
2,14.607743
3,19.784810
4,17.932895
5,18.717949
6,20.355932
7,29.989130
8,15.704918
9,1.730769
